In [196]:
import pandas as pd
import numpy as np

In [197]:
dataset = pd.read_csv("AmazonData.csv")
dataset.head(5)

,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,...,Product Url,Stock,Product Details,Dimensions,Color,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description
0,4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",NaN,NaN,Sports & Outdoors | Outdoor Recreation | Skate...,NaN,NaN,$237.68,NaN,NaN,...,https://www.amazon.com/DB-Longboards-CoreFlex-...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
1,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",NaN,NaN,Toys & Games | Learning & Education | Science ...,NaN,NaN,$99.95,NaN,55324,...,https://www.amazon.com/Electronic-Circuits-Cla...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,NaN,NaN,Toys & Games | Arts & Crafts | Craft Kits,NaN,NaN,$34.99,NaN,NaN,...,https://www.amazon.com/3Doodler-Plastic-Innova...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
3,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,NaN,NaN,Toys & Games | Hobbies | Models & Model Kits |...,NaN,NaN,$28.91,NaN,142,...,https://www.amazon.com/Guillow-Airplane-Design...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
4,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,NaN,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,NaN,NaN,$17.49,NaN,62151,...,https://www.amazon.com/Woodstock-Collage-500-p...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


In [198]:
dataset.shape

(10002, 28)

In [199]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Uniq Id                10002 non-null  object 
 1   Product Name           10002 non-null  object 
 2   Brand Name             0 non-null      float64
 3   Asin                   0 non-null      float64
 4   Category               9172 non-null   object 
 5   Upc Ean Code           34 non-null     object 
 6   List Price             0 non-null      float64
 7   Selling Price          9895 non-null   object 
 8   Quantity               0 non-null      float64
 9   Model Number           8232 non-null   object 
 10  About Product          9729 non-null   object 
 11  Product Specification  8370 non-null   object 
 12  Technical Details      9212 non-null   object 
 13  Shipping Weight        8864 non-null   object 
 14  Product Dimensions     479 non-null    object 
 15  Im

In [200]:
dataset.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
9997     False
9998     False
9999     False
10000    False
10001    False
Length: 10002, dtype: bool

In [201]:
cols = [0,2,3,5,6,8,9,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
dataset.drop(dataset.columns[cols], axis =1, inplace=True)

In [202]:
dataset.dropna(inplace = True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7216 entries, 0 to 10001
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Product Name           7216 non-null   object
 1   Category               7216 non-null   object
 2   Selling Price          7216 non-null   object
 3   About Product          7216 non-null   object
 4   Product Specification  7216 non-null   object
 5   Shipping Weight        7216 non-null   object
dtypes: object(6)
memory usage: 394.6+ KB


In [203]:
new = dataset["Category"].str.split("|", n = 3, expand = True)

new = new.apply(lambda x: x.str.strip())
  
# making the first category called Main Category
dataset["Main Category"]= new[0] 
  
# making the second category called sub_category 
dataset["Sub Category"]= new[1]

# making the third category called side_category 
dataset["Side Category"]= new[2]

# making the last column consist of the remaining categories
dataset["Other Categories"]= new[3]

# Dropping old category columns and the remaining categories 
dataset.drop(columns =["Category", "Other Categories"], inplace = True)

In [204]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7216 entries, 0 to 10001
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Product Name           7216 non-null   object
 1   Selling Price          7216 non-null   object
 2   About Product          7216 non-null   object
 3   Product Specification  7216 non-null   object
 4   Shipping Weight        7216 non-null   object
 5   Main Category          7216 non-null   object
 6   Sub Category           7216 non-null   object
 7   Side Category          6231 non-null   object
dtypes: object(8)
memory usage: 507.4+ KB


In [205]:
dataset['Main Category'] = dataset['Main Category'].astype('string')
dataset['Sub Category'] = dataset['Sub Category'].astype('string')

In [206]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7216 entries, 0 to 10001
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Product Name           7216 non-null   object
 1   Selling Price          7216 non-null   object
 2   About Product          7216 non-null   object
 3   Product Specification  7216 non-null   object
 4   Shipping Weight        7216 non-null   object
 5   Main Category          7216 non-null   string
 6   Sub Category           7216 non-null   string
 7   Side Category          6231 non-null   object
dtypes: object(6), string(2)
memory usage: 507.4+ KB


In [207]:
dataset["Main Category"]

0        Sports & Outdoors
1             Toys & Games
2             Toys & Games
3             Toys & Games
4             Toys & Games
               ...        
9995        Home & Kitchen
9996          Toys & Games
9998          Toys & Games
9999       Office Products
10001       Home & Kitchen
Name: Main Category, Length: 7216, dtype: string

In [208]:
dataset["Sub Category"]

0              Outdoor Recreation
1            Learning & Education
2                   Arts & Crafts
3                         Hobbies
4                         Puzzles
                   ...           
9995                      Bedding
9996                Building Toys
9998                Arts & Crafts
9999     Office & School Supplies
10001                   Furniture
Name: Sub Category, Length: 7216, dtype: string

In [209]:
main_category_input = 'Toys & Games'
sub_category_input = 'Building Toys'

In [210]:
dataset['Main Category'].unique()

<StringArray>
[                       'Sports & Outdoors',
                             'Toys & Games',
                'Clothing, Shoes & Jewelry',
                           'Home & Kitchen',
                             'Pet Supplies',
                    'Arts, Crafts & Sewing',
                              'Electronics',
                     'Patio, Lawn & Garden',
                 'Tools & Home Improvement',
                          'Office Products',
                            'Baby Products',
                   'Grocery & Gourmet Food',
                              'Video Games',
    'Remote & App Controlled Vehicle Parts',
                       'Health & Household',
                  'Industrial & Scientific',
                   'Beauty & Personal Care',
                               'Automotive',
 'Remote & App Controlled Vehicles & Parts',
                      'Musical Instruments',
                                  'Hobbies',
                              'Movies & T

In [211]:
dataset['Sub Category'].unique()

<StringArray>
[                      'Outdoor Recreation',
                     'Learning & Education',
                            'Arts & Crafts',
                                  'Hobbies',
                                  'Puzzles',
                   'Costumes & Accessories',
                               'Home Décor',
                      'Baby & Toddler Toys',
                         'Collectible Toys',
                            'Building Toys',
                                'Furniture',
                       'Novelty & Gag Toys',
                    'Sports & Outdoor Play',
                        "Kids' Electronics",
                         'Sports & Fitness',
                  'Dress Up & Pretend Play',
                                  'Bedding',
                   'Toy Figures & Playsets',
                           'Party Supplies',
                      'Dolls & Accessories',
                      'Games & Accessories',
                                     'Dog

In [212]:
# Verifica se as categorias fornecidas são válidas
if main_category_input not in dataset['Main Category'].unique():
    print('Categoria principal não encontrada na base de dados.')
if sub_category_input not in dataset['Sub Category'].unique():
    print('Subcategoria não encontrada na base de dados.')


In [213]:

# Calcula uma pontuação para cada produto com base na similaridade das categorias
dataset['score'] = 0

# Aumenta a pontuação se a "Main Category" ou "Sub Category" corresponder
dataset.loc[dataset['Main Category'] == main_category_input, 'score'] += 2
dataset.loc[dataset['Sub Category'] == sub_category_input, 'score'] += 1


In [215]:

# Ordena os produtos com base na pontuação, do mais alto para o mais baixo
produtos_recomendados = dataset.sort_values(by='score', ascending=False)

# Remove produtos com pontuação 0 (sem correspondência)
produtos_recomendados = produtos_recomendados[produtos_recomendados['score'] > 0]

# Remove a coluna de pontuação antes de retornar
produtos_recomendados = produtos_recomendados.drop(columns='score')

# Retorna as recomendações
if produtos_recomendados.empty:
    print('Nenhuma recomendação encontrada para essas categorias.')

print(produtos_recomendados)

3349                    LEGO Storage Brick 8, Bright Pink
1580    Fischertechnik Dynamic XS Building Kit (70 Piece)
2593    Magformers Sweet House 64 Pieces Pink and Purp...
9432    Ravensburger Gravitrax Flip Accessory - Marble...
3470    Plus-Plus BIG - Preschool Construction Buildin...
                              ...                        
2627    Manhattan Toy Natural Historian Opposites Padd...
9470       Squadron Products Tools Rubber Block Model Kit
9424                            Runewars: Essentials Pack
9423    Harvey Ball 2-Tier Lacquer Bento Lunch Box wit...
2880    Calypso Studios Inc. 3" Comforting Clay Cross ...
Name: Product Name, Length: 5556, dtype: object
